<a href="https://colab.research.google.com/github/dkimds/HF-Transformer/blob/main/Llama3p2_fine_tuning_korquad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KorQuAD v1.0 데이터 다운로드(dev)

In [ ]:
!wget https://raw.githubusercontent.com/korquad/korquad.github.io/master/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

In [ ]:
import json

with open('KorQuAD_v1.0_dev.json', 'r') as f:
    dev_data = json.load(f)

dev_data = [item for topic in dev_data['data'] for item in topic['paragraphs'] ]

# KorQuad 데이터셋을 Llama2 학습을 위한 Prompt 형태로 정제


In [ ]:
refined_dict = {}
for original_dict in dev_data:
  for entry in original_dict['qas']:
      question = entry['question']
      answers = [ans['text'] for ans in entry['answers']][0]
      refined_dict[question] = answers

print(refined_dict)

In [ ]:
# 빠른 학습을 위해 20개만 추출
num_items = 20

final_prompt_list = []
for idx, (question, answer) in enumerate(refined_dict.items()):
  if idx >= num_items:
    break
  prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: {question} ### Response: {answer}"
  print(idx, prompt)
  prompt_dict = {}
  prompt_dict['text'] = prompt
  final_prompt_list.append(prompt_dict)

In [ ]:
final_prompt_list

# JSON 파일로 저장 및 다운로드

In [ ]:
with open('korquad_data.json', 'w', encoding='utf-8') as f:
    f.write("[\n")
    for i, item in enumerate(final_prompt_list):
        json_str = json.dumps(item, ensure_ascii=False)
        f.write(json_str)
        if i < len(final_prompt_list) - 1:  # 마지막 원소가 아니라면 콤마를 추가
            f.write(",\n")
        else:  # 마지막 원소라면 콤마를 추가하지 않음
            f.write("\n")
    f.write("]\n")

In [ ]:
from google.colab import files
files.download('korquad_data.json')

# ChatGPT를 이용한 Data Augmentation

In [ ]:
# ChatGPT를 이용해서 생성한 10개의 Data Augmentation을 위한 문장들
augmentation_dict = {}
# idx 0
augmentation_dict["임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?"] = [
"임종석은 여의도 농민 폭력 시위를 주도한 혐의로 언제 지명수배되었나요?",
"여의도 농민 폭력 시위를 주도한 혐의로 임종석이 지명수배된 날짜는?",
"임종석이 여의도 농민 폭력 시위 주도 혐의로 지명수배 받은 날은 언제인가요?",
"여의도 농민 폭력 시위 주도 혐의로 지명수배된 임종석의 그 날은?",
"언제 임종석은 여의도 농민 폭력 시위를 주도한 혐의로 지명수배되었나요?",
"임종석이 여의도 농민 폭력 시위의 주도자로 지목되어 지명수배된 날은 언제인가요?",
"임종석이 여의도 농민 폭력 시위를 이끌었다는 혐의로 언제 지명수배를 당했나요?",
"여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 임종석의 지명수배 날짜는?",
"여의도 농민 폭력 시위 주도 혐의로 임종석이 언제 지명수배를 받았는지?",
"임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배를 받은 날짜는 언제인가요?"
]
# idx 1
augmentation_dict["1989년 6월 30일 평양축전에 대표로 파견 된 인물은?"] = [
"1989년 6월 30일에 평양축전 대표로 파견된 사람은 누구인가요?",
"평양축전에 대표로 보내진 인물이 누군지, 그 날짜가 1989년 6월 30일입니다.",
"1989년 6월 30일에 평양축전에 대표로 간 사람은 누구인가요?",
"1989년 6월 30일, 평양축전의 대표로 파견된 인물은 누구였나요?",
"평양축전에 대표로 출장한 인물이 누구인지, 그 날은 1989년 6월 30일입니다.",
"1989년 6월 30일에 평양축전을 대표로 참석한 인물은 누구였습니까?",
"1989년 6월 30일에 대표로 평양축전에 참가한 사람은 누구였나요?",
"1989년 6월 30일, 평양축전 대표로 파견된 사람은 누구인가요?",
"누가 1989년 6월 30일에 평양축전의 대표로 파견되었나요?",
"평양축전에 대표로 파견된 인물은 누구였고, 그 일자는 1989년 6월 30일입니다."
]
# idx 2
augmentation_dict["임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?"] = [
"임종석이 여의도 농민 폭력 시위 주도 혐의로 지명수배 받은 연도는 언제인가요?",
"여의도 농민 폭력 시위를 주도한 혐의로 임종석이 지명수배된 년도는?",
"임종석이 여의도 농민 폭력 시위를 이끌었다는 혐의로 어느 연도에 지명수배되었나요?",
"언제 임종석은 여의도 농민 폭력 시위를 주도한 혐의로 지명수배를 받았나요?",
"여의도 농민 폭력 시위 주도 혐의로 임종석이 지명수배된 연도는 언제인가요?",
"임종석이 여의도 농민 폭력 시위의 주도자로 지목되어 지명수배된 연도는 어느 해인가요?",
"여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 임종석의 지명수배 연도는?",
"여의도 농민 폭력 시위 주도 혐의로 임종석이 어느 연도에 지명수배를 받았는지?",
"임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배를 받은 연도는 언제입니까?",
"임종석이 여의도 농민 폭력 시위의 주도자로 혐의를 받아 지명수배된 년도는 언제인가요?"
]
# idx 3
augmentation_dict["임종석을 검거한 장소는 경희대 내 어디인가?"] = [
"임종석이 어디에서 검거되었는지, 그 장소는 경희대 내부입니다.",
"경희대학교 내에서 임종석은 어디서 검거되었나요?",
"경희대 내부에서 임종석이 검거된 위치는 어디인가요?",
"임종석이 경희대 내에서 어느 위치에서 검거되었습니까?",
"경희대학교 내부의 어느 위치에서 임종석은 검거되었나요?",
"임종석의 검거 장소는 경희대 내 어느 부분인가요?",
"임종석이 경희대 내에서 검거된 구체적인 장소는 어디입니까?",
"어느 위치에서 임종석이 경희대학교 내부에서 검거되었나요?",
"임종석이 경희대학교 내에서 검거된 지점은 어디인가요?",
"임종석을 경희대 내에서 검거한 구체적인 장소는 어디인가요?"
]
# idx 4
augmentation_dict["임종석이 조사를 받은 뒤 인계된 곳은 어딘가?"] = [
"임종석이 조사 후에 어디로 인계되었나요?",
"조사를 마친 후 임종석이 인계된 장소는 어디인가요?",
"임종석이 조사를 받고 난 뒤 전달된 위치는 어디입니다?",
"조사를 받은 이후에 임종석이 이송된 곳은 어디였나요?",
"임종석은 조사를 마치고 어느 장소로 인계되었습니까?",
"어디로 임종석이 조사 후 인계되었는지?",
"임종석이 수사를 받은 후에 이동한 곳은 어디입니까?",
"임종석이 조사를 완료한 뒤 전달된 장소는 어디였습니까?",
"조사가 끝난 후에 임종석은 어디로 인계되었나요?",
"임종석이 조사를 받고 난 이후 이송된 장소는 어디인가요?"
]
# idx 5
augmentation_dict["1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?"] = [
"1989년 2월 15일에 여의도 농민 폭력 시위 주도 혐의로 지명수배된 사람은 누구인가요?",
"여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은 무엇이며, 그 일자는 1989년 2월 15일입니다.",
"1989년 2월 15일에 여의도 농민 폭력 시위를 이끌었다는 혐의로 지명수배된 인물은 누구인가요?",
"1989년 2월 15일, 여의도 농민 폭력 시위의 주도자로 지명수배된 사람의 이름은?",
"여의도 농민 폭력 시위를 주도한 혐의로 1989년 2월 15일에 지명수배된 사람은 누구였나요?",
"누가 1989년 2월 15일에 여의도 농민 폭력 시위를 주도한 혐의로 지명수배되었나요?",
"1989년 2월 15일에 여의도 농민 폭력 시위 주도 혐의로 지명수배를 받은 사람의 이름은 어떻게 되나요?",
"1989년 2월 15일에 여의도 농민 폭력 시위의 주도자로 혐의를 받아 지명수배된 사람은 누구였습니까?",
"1989년 2월 15일, 여의도 농민 폭력 시위를 주도한 혐의로 지명수배를 받은 인물의 이름은 무엇인가요?",
"여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 인물이 누구인지, 그 날짜가 1989년 2월 15일입니다."
]
# idx 6
augmentation_dict["임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?"] = [
"1989년 2월 15일에 지명수배 받은 임종석의 혐의는 어떤 시위를 주도했다는 것입니다?",
"임종석이 어떤 시위를 주도한 혐의로 1989년 2월 15일에 지명수배되었나요?",
"1989년 2월 15일, 임종석은 어떤 시위를 주도한 혐의로 지명수배를 받았습니까?",
"어떤 시위를 주도했다는 혐의로 임종석이 1989년 2월 15일에 지명수배되었나요?",
"1989년 2월 15일에 임종석이 지명수배를 받은 이유는 어떤 시위를 주도했다는 것인가요?",
"임종석이 1989년 2월 15일에 어떤 시위 주도 혐의로 지명수배되었는지?",
"1989년 2월 15일 임종석이 지명수배를 받은 이유는 어떤 시위를 이끌었다는 것인가?",
"1989년 2월 15일에 어떤 시위를 이끈 혐의로 임종석이 지명수배되었습니까?",
"임종석이 1989년 2월 15일에 지명수배 받은 것은 어떤 시위를 주도했다는 혐의입니다. 맞나요?",
"임종석이 1989년 2월 15일에 지명수배 받은 혐의와 관련된 주도한 시위는 무엇인가요?"
]
# idx 7
augmentation_dict["정부의 헌법개정안 준비 과정에 대해서 청와대 비서실이 아니라 국무회의 중심으로 이뤄졌어야 했다고 지적한 원로 헌법학자는?"] = [
"헌법개정안의 준비 과정이 국무회의 중심으로 이뤄져야 한다고 지적한 원로 헌법학자는 누구인가?",
"국무회의가 헌법개정안 준비 과정의 중심이 되어야 한다고 주장한 원로 헌법학자는 누구인가요?",
"청와대 비서실 대신 국무회의가 헌법개정안을 주도해야 한다고 지적한 원로 헌법학자의 이름은?",
"누가 헌법개정안의 준비 과정에 대해 국무회의 중심으로 이뤄져야 한다고 주장했나요?",
"원로 헌법학자 중에서 헌법개정안 준비 과정이 국무회의를 중심으로 이뤄져야 한다고 지적한 사람은 누구입니까?",
"헌법개정안을 준비하는 과정에서 국무회의가 중심이 되어야 한다고 언급한 원로 헌법학자는 누구인가요?",
"헌법개정안의 준비 과정에 대해 국무회의 중심으로 진행되어야 한다고 주장한 원로 헌법학자는?",
"국무회의가 헌법개정안의 준비 과정에서 중심 역할을 해야 한다고 주장한 원로 헌법학자의 이름은 무엇인가요?",
"누구는 헌법개정안 준비 과정에 대해 청와대 비서실이 아니라 국무회의가 중심이 되어야 한다고 지적했나요?",
"국무회의 중심의 헌법개정안 준비 과정이 필요하다고 지적한 원로 헌법학자는 누구입니까?"
]
# idx 8
augmentation_dict["'행보가 비서 본연의 역할을 벗어난다', '장관들과 내각이 소외되고 대통령비서실의 권한이 너무 크다'는 의견이 제기된 대표적인 예는?"] = [
"비서의 역할을 벗어난 행동과 대통령비서실의 과도한 권한 때문에 장관과 내각이 소외된 대표적인 사례는 무엇인가?",
"장관들과 내각이 대통령비서실의 큰 권한에 의해 소외되고, 비서의 행동이 본연의 역할을 넘어선 대표적인 예는 어떤 경우인가?",
"대통령비서실의 권한 확대와 비서의 역할 초과가 문제로 지적되는 대표적인 사례는 무엇인가요?",
"대통령비서실의 과도한 권한과 비서의 본연의 역할을 넘어선 행동에 대한 대표적인 비판의 예는 어떤 것인가?'",
"장관과 내각이 소외되는 등, 대통령비서실의 권한이 너무 크고 비서의 역할이 넘어선 대표적인 경우는 무엇인가?",
"대통령비서실이 과도한 권한을 행사하고, 비서가 본연의 역할을 벗어나는 대표적인 예시는 무엇인가요?",
"비서가 본연의 역할을 초과하고 대통령비서실의 권한이 과도하게 크다고 지적된 대표적인 사례는 어떤 것인가요?",
"'비서의 역할을 넘어서는 행동'과 '장관과 내각의 소외'가 문제로 제기된 대표적인 예는 무엇인가?",
"본연의 역할을 벗어난 비서의 행동과 과도한 대통령비서실의 권한이 지적되는 대표적인 사례는?",
"장관들과 내각이 대통령비서실에 의해 소외되고, 비서의 행동이 본연의 역할을 벗어나는 대표적인 사례는 어떤 것인가?"
]
# idx 9
augmentation_dict["국무회의의 심의를 거쳐야 한다는 헌법 제 몇 조의 내용인가?"] = [
"헌법에서 국무회의의 심의가 필요하다고 명시된 조항은 어떤 것인가?",
"국무회의 심의가 요구되는 헌법의 조항은 제 몇 조인가요?",
"헌법 제 몇 조에서는 국무회의의 심의를 필요로 한다고 되어있나요?",
"국무회의의 심의가 필수적이라고 헌법에서 규정한 조항은 어떤 것인가?",
"국무회의 심의가 헌법에서 어떤 조항에 의해 요구되는가?",
"헌법에서 국무회의의 심의를 명시한 조항은 제 몇 조인가?",
"어떤 헌법 조항이 국무회의의 심의를 필요로 하나요?",
"국무회의 심의가 헌법에 의해 요구되는 조항은 어떤 것인가요?",
"헌법의 어느 조항에서 국무회의의 심의가 필요하다고 되어있나요?",
"헌법에서 국무회의의 심의가 규정된 조항은 어느 것인가요?"
]
# idx 10
augmentation_dict["법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다고 지적한 경희대 석좌교수 이름은?"] = [
"법무부 장관 대신 민정수석이 개정안을 소개하는 것에 대해 이해할 수 없다고 말한 경희대 석좌교수는 누구인가?",
"민정수석이 법무부 장관을 무시하고 개정안을 설명하는 것이 논리적으로 이해되지 않는다고 경희대 석좌교수가 지적한 사람은?",
"민정수석이 법무부 장관을 건너뛰고 개정안을 발표하는 것이 이해가 가지 않는다고 경희대의 석좌교수가 언급한 이름은?",
"경희대 석좌교수가 민정수석이 법무부 장관을 무시하고 개정안을 설명하는 것은 이해가 안 간다고 지적한 사람은 누구인가?",
"민정수석이 법무부 장관을 제외하고 개정안을 발표하는 것에 문제가 있다고 지적한 경희대 석좌교수의 이름은?",
"법무부 장관을 무시하고 민정수석이 개정안을 주선하는 것이 이해가 안 된다고 경희대 석좌교수가 주장한 사람은 누구인가?",
"민정수석이 법무부 장관 대신에 개정안을 소개하는 것이 논리에 맞지 않다고 경희대 석좌교수가 언급한 이름은?",
"민정수석이 개정안을 설명하는 것에 대해 법무부 장관을 무시하는 것은 이해할 수 없다고 지적한 경희대 석좌교수는 누구인가요?",
"법무부 장관 대신에 민정수석이 개정안을 발표하는 것에 대한 비판을 한 경희대 석좌교수의 이름은?",
"경희대 석좌교수가 법무부 장관을 무시하고 민정수석이 개정안을 소개하는 것은 이해가 안 된다고 지적한 이는 누구인가?"
]
# idx 11
augmentation_dict["미국 군대 내 두번째로 높은 직위는 무엇인가?"] = [
"미국 군대에서 두 번째로 높은 계급은 어떤 것인가요?",
"미국 군에서 최고 계급 바로 아래의 직위는 무엇인가?",
"미국의 군대에서 가장 높은 직위 다음으로 높은 직위는 무엇인가요?",
"미국 군에서 두 번째로 높은 직책은 어떤 것인가?",
"미국 군대에서 최상위 계급에 이은 두 번째 계급은 무엇인가?",
"미국 군대 내에서 두 번째로 높은 계급이 무엇인지 알려주실 수 있나요?",
"미국 군대의 두 번째로 높은 계급이 무엇인지 알고 싶습니다.",
"미국 군에서 가장 높은 직위에 이은 두 번째 직위는 무엇인가요?",
"미국 군대에서 최고 계급을 제외한 다음으로 높은 계급은 무엇인가요?",
"미국의 군대에서 두 번째로 높은 계급 또는 직위는 무엇인가요?"
]
# idx 12
augmentation_dict["로널드 레이건 정부 출범 당시 알렉산더 헤이그는 어떤 직책을 맡았는가?"] = [
"로널드 레이건 대통령이 취임했을 때 알렉산더 헤이그의 직책은 무엇이었나요?",
"알렉산더 헤이그가 로널드 레이건 정부가 시작될 때 어떤 역할을 맡았는지 알려주실 수 있나요?",
"로널드 레이건의 정부가 취임한 시점에서 알렉산더 헤이그는 어떤 직위에 있었나요?",
"로널드 레이건 대통령 취임 시, 알렉산더 헤이그가 담당한 직책은 무엇이었는가?",
"로널드 레이건 정부가 출범했을 때, 알렉산더 헤이그의 역할은 무엇이었나요?",
"로널드 레이건 대통령 취임 초기에 알렉산더 헤이그는 어떤 직책을 수행했나요?",
"로널드 레이건 정부가 처음 시작됐을 때 알렉산더 헤이그는 어떤 포지션을 가졌나요?",
"로널드 레이건 대통령 출범 때 알렉산더 헤이그의 직책은 무엇인가요?",
"알렉산더 헤이그는 로널드 레이건 대통령이 취임했을 때 어떤 직을 맡았는가?",
"로널드 레이건 정부의 초기에 알렉산더 헤이그가 가진 직책은 무엇이었나요?"
]
# idx 13
augmentation_dict["알렉산더 헤이그는 어느 대통령의 밑에서 국무장관을 지냈는가?"] = [
"어느 대통령 아래에서 알렉산더 헤이그는 국무장관을 역임했나요?",
"알렉산더 헤이그가 국무장관을 맡았던 대통령은 누구인가요?",
"알렉산더 헤이그는 어떤 대통령 시기에 국무장관을 지냈나요?",
"알렉산더 헤이그가 국무장관을 서한 대통령은 누구였나요?",
"알렉산더 헤이그는 국무장관을 어느 대통령 아래에서 수행했는가?",
"알렉산더 헤이그가 국무장관으로 재임한 대통령은 누구였습니까?",
"어느 대통령의 정부에서 알렉산더 헤이그는 국무장관을 지냈습니까?",
"알렉산더 헤이그는 누구의 대통령 시절에 국무장관을 역임했나요?",
"알렉산더 헤이그의 국무장관 재임 시점의 대통령은 누구였나요?",
"알렉산더 헤이그가 국무장관을 지낸 대통령은 어느 분이었나요?"
]
# idx 14
augmentation_dict["로널드 레이건 대통령 밑에서 일한 국무 장관은 누구인가?"] = [
"로널드 레이건 대통령 아래에서 국무장관을 지낸 사람은 누구였나요?",
"로널드 레이건 대통령 시절의 국무장관은 누구인가요?",
"로널드 레이건 대통령이 재임할 때의 국무장관은 누구였습니까?",
"로널드 레이건 대통령의 국무장관은 누구였나요?",
"로널드 레이건 대통령 아래에서 국무장관을 역임한 사람은 누구인가?",
"로널드 레이건 대통령 시절에 국무장관을 맡았던 인물은 누구인가요?",
"로널드 레이건 대통령이 재임 중에 어떤 사람이 국무장관을 지냈나요?",
"로널드 레이건 대통령과 함께 일한 국무장관은 누구였을까요?",
"로널드 레이건 대통령의 정부에서 국무장관 역할을 한 사람은 누구인가?",
"로널드 레이건 대통령이 재임하던 시기의 국무장관은 누구였나요?"
]
# idx 15
augmentation_dict["미국 군대에서 두번째로 높은 직위는?"] = [
"미국 군대의 두번째로 높은 계급은 무엇인가요?",
"미국 군대에서 가장 높은 직위 다음으로 높은 직위는 무엇인가요?",
"미국 군대의 계급 중에서 두번째로 높은 것은 무엇입니까?",
"미국 군대에서 최고 직위 바로 아래에 있는 직위는 무엇입니까?",
"미국 군대의 두번째 높은 계급은 무엇으로 알려져 있나요?",
"미국 군대에서 최상위 계급 다음에 오는 계급은 어떤 것인가요?",
"미국 군대에서 두번째로 높은 계급이 무엇인지 알려주세요.",
"미국 군대의 두번째로 높은 직책은 무엇입니까?",
"미국 군대에서 두번째로 높은 직책이 무엇인가요?",
"미국 군대의 계급 중 두번째로 높은 것은 어떤 것인가요?"
]
# idx 16
augmentation_dict["알렉산더 메이그스 헤이그의 생년월일은?"] = [
"알렉산더 메이그스 헤이그은 언제 태어났나요?",
"알렉산더 메이그스 헤이그의 출생일은 언제인가요?",
"알렉산더 메이그스 헤이그의 탄생일은 언제입니까?",
"알렉산더 메이그스 헤이그의 생일은 어느 날인가요?",
"알렉산더 메이그스 헤이그이 언제 출생했나요?",
"알렉산더 메이그스 헤이그의 출생 날짜는 언제입니까?",
"알렉산더 메이그스 헤이그이 언제 태어난 것인가요?",
"알렉산더 메이그스 헤이그은 어느 날 태어났습니까?",
"알렉산더 메이그스 헤이그의 탄생 날짜는 무엇인가요?",
"알렉산더 메이그스 헤이그의 생일은 언제로 알려져 있나요?"
]
# idx 17
augmentation_dict["알렉산더 헤이그가 로널드 레이건 대통령 밑에서 맡은 직책은 무엇이었나?"] = [
"알렉산더 헤이그은 로널드 레이건 정부에서 어떤 역할을 했나요?",
"알렉산더 헤이그가 로널드 레이건 대통령 아래에서 담당한 직책은 무엇이었습니까?",
"로널드 레이건 대통령 시절에 알렉산더 헤이그는 어떤 직위에 있었나요?",
"로널드 레이건 대통령 밑에서 알렉산더 헤이그의 직책은 무엇이었습니까?",
"알렉산더 헤이그가 로널드 레이건 대통령과 함께 일할 때 그의 직책은 무엇이었나요?",
"알렉산더 헤이그는 로널드 레이건 대통령 아래에서 어떤 직책을 맡았나요?",
"로널드 레이건 대통령의 정부에서 알렉산더 헤이그의 역할은 무엇이었나요?",
"알렉산더 헤이그가 로널드 레이건 대통령의 정부에서 담당한 역할은 무엇이었나요?",
"로널드 레이건 대통령 아래에서 알렉산더 헤이그는 어떤 역할을 했습니까?",
"알렉산더 헤이그이 로널드 레이건 대통령 시절에 어떤 직책을 가졌나요?"
]
# idx 18
augmentation_dict["알렉산더 헤이그가 1984년 발간한 회고록의 제목은 무엇인가?"] = [
"1984년에 알렉산더 헤이그이 출판한 회고록의 이름은 무엇입니까?",
"알렉산더 헤이그의 1984년에 발표한 회고록의 제목은 어떻게 되나요?",
"알렉산더 헤이그이 1984년에 낸 회고록은 어떤 제목을 가지고 있나요?",
"알렉산더 헤이그의 1984년 회고록은 무슨 제목으로 알려져 있나요?",
"1984년에 알렉산더 헤이그이 작성한 회고록의 제목은 무엇으로 알려져 있습니까?",
"알렉산더 헤이그이 1984년에 발간한 자신의 회고록의 제목은 무엇인가요?",
"알렉산더 헤이그의 1984년 작성된 회고록의 제목은 무엇으로 되어 있나요?",
"알렉산더 헤이그이 1984년에 출판한 그의 회고록은 어떤 제목을 가졌나요?",
"1984년 알렉산더 헤이그의 회고록 제목은 무엇이었나요?",
"알렉산더 헤이그이 1984년에 작성하고 발간한 회고록의 제목은 어떤 것인가요?"
]
# idx 19
augmentation_dict["알렉산더 헤이그와 1950년 5월 결혼한 상대의 이름은 무엇인가?"] = [
"1950년 5월에 알렉산더 헤이그이 결혼한 사람의 이름은 무엇입니까?",
"알렉산더 헤이그이 1950년 5월에 결혼한 상대방의 이름은 어떻게 되나요?",
"알렉산더 헤이그의 1950년 5월 결혼 파트너의 이름은 무엇인가요?",
"알렉산더 헤이그와 1950년 5월에 청혼한 사람의 명칭은 무엇으로 알려져 있나요?",
"1950년 5월에 알렉산더 헤이그이와 결혼한 인물의 이름은 무엇으로 알려져 있습니까?",
"알렉산더 헤이그와 1950년 5월에 맺은 혼인의 상대는 누구인가요?",
"알렉산더 헤이그이 1950년 5월에 결혼한 사람은 누구로 알려져 있나요?",
"1950년 5월, 알렉산더 헤이그와 혼인을 맺은 사람의 이름은 무엇이었나요?",
"알렉산더 헤이그이 1950년 5월에 결혼한 여성 또는 남성의 이름은 무엇인가요?",
"1950년 5월에 알렉산더 헤이그와 웨딩을 올린 상대방의 이름은 무엇인가요?"
]

In [ ]:
# 빠른 학습을 위해 20개만 추출
num_items = 20

final_prompt_list = []
for idx, (question, answer) in enumerate(refined_dict.items()):
  if idx >= num_items:
    break
  print(question)
  prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: {question} ### Response: {answer}"
  print(idx, prompt)
  prompt_dict = {}
  prompt_dict['text'] = prompt
  final_prompt_list.append(prompt_dict)
  # Data Augmentation
  auagmentated_questions = augmentation_dict[question]
  print(len(auagmentated_questions))
  for each_question in auagmentated_questions:
    prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: {each_question} ### Response: {answer}"
    print(idx, prompt)
    prompt_dict = {}
    prompt_dict['text'] = prompt
    final_prompt_list.append(prompt_dict)

In [ ]:
with open('korquad_data_augmented.json', 'w', encoding='utf-8') as f:
    f.write("[\n")
    for i, item in enumerate(final_prompt_list):
        json_str = json.dumps(item, ensure_ascii=False)
        f.write(json_str)
        if i < len(final_prompt_list) - 1:  # 마지막 원소가 아니라면 콤마를 추가
            f.write(",\n")
        else:  # 마지막 원소라면 콤마를 추가하지 않음
            f.write("\n")
    f.write("]\n")

In [ ]:
from google.colab import files
files.download('korquad_data_augmented.json')

# Llama 3를 KorQuad 데이터셋에 맞게 fine-tuning 하기

In [ ]:
!nvidia-smi

## AutoTrain Advanced
https://github.com/huggingface/autotrain-advanced

In [ ]:
!pip install -q autotrain-advanced

## PyTorch 업데이트

In [ ]:
!autotrain setup --update-torch

In [ ]:
!pip install bitsandbytes --upgrade --quiet

# HuggingFace Access Token 설정

In [ ]:
import os
os.environ['HF_TOKEN']="..."

## Korquad 데이터셋에 맞게 Llama3 Fine-Tuning

- autotrain 설정값: https://github.com/huggingface/autotrain-advanced/blob/f1367b590dfc53d240e9684779991da540590386/src/autotrain/cli/run_llm.py#L21 (**과거 버전[0.6.35]**)
- autotrain 설정값: https://github.com/huggingface/autotrain-advanced/blob/main/src/autotrain/cli/run_llm.py#L17 (**최근 버전[0.6.80]**)


In [ ]:
# Data Augmentation 적용 o
!autotrain llm --train \
    --project-name "llama3p2-korquad-finetuning-da-3B" \
    --model "meta-llama/Llama-3.2-3B" \
    --data-path "korquad_prompt_da" \
    --text-column "text" \
    --peft \
    --lr 2e-4 \
    --batch-size 8 \
    --epochs 40 \
    --trainer sft \
    --model_max_length 256

# 학습결과 zip 파일로 압축후 다운로드

In [ ]:
import zipfile
import shutil
from google.colab import files

# 압축할 폴더 이름
# folder_name = "llama3p2-korquad-finetuning-da-1B"  # Data Augmentation 적용 o
folder_name = "llama3p2-korquad-finetuning-da-3B"  # Data Augmentation 적용 o

# 생성될 ZIP 파일 이름
# zip_file_name = "llama3p2-korquad-finetuning-da-1B.zip" # Data Augmentation 적용 o
zip_file_name = "llama3p2-korquad-finetuning-da-3B.zip" # Data Augmentation 적용 o

# 폴더를 ZIP 파일로 압축
shutil.make_archive(zip_file_name[:-4], 'zip', folder_name)

# ZIP 파일을 로컬로 다운로드
files.download(zip_file_name)

# 학습결과 불러오기

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

# 베이스 모델을 파인튜닝한 모델과 동일하게 변경
# model_id = "meta-llama/Llama-3.2-1B"  # 이것으로 변경!
# peft_model_id = "./llama3p2-korquad-finetuning-da-1B"
model_id = "meta-llama/Llama-3.2-3B"  # 이것으로 변경!
peft_model_id = "./llama3p2-korquad-finetuning-da-3B"
config = PeftConfig.from_pretrained(peft_model_id)

# 1B(3B) 모델이라 quantization 필요 없음
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 패딩 토큰 설정 (필요시)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model.eval()

In [ ]:
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: %s ### Response: "

# do_sample : False -> 항상 가장 높은 확률값을 가진 단어로 예측
def gen(x):
    q = prompt % (x,)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=128,
        early_stopping=True,
        do_sample=False,
    )
    return tokenizer.decode(gened[0]).replace(q, "")

# 20개 sample에 대한 inference (Fine-Tuning된 모델 성능 테스트)

In [ ]:
# idx 0
gen("임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?")
# 정답 : 1989년 2월 15일
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 임종석은 1989년 5월 26일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배되었습니다. \
# 그는 이후 1995년 5월 18일에 체포되었고, 여러 차례의 재판 끝에 1996년에 석방되었습니다.
# 이 사건은 당시 대한민국 사회와 정치에 큰 영향을 끼쳤으며, 임종석 자신도 이후에는 정치와 사회운동, 교육 등 여러 분야에서 활동하게 됩니다.
# 참고로 정보가 2021년 9월까지 업데이트되어 있으니, 그 이후의 정보는 확인할 수 없습니다.

# GPT-4 맞춤유무 : x

In [ ]:
# idx 1
gen("1989년 6월 30일 평양축전에 대표로 파견 된 인물은?")
# 정답 : 임수경
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 1989년 6월 30일에 열린 평양축전(World Festival of Youth and Students)에 대한 정확한 정보는 제한되어 있지만,
# 이 이벤트는 북한 평양에서 열린 것으로 알려져 있습니다. 이 축전은 여러 나라의 청년들이 모여 문화 교류와 평화를 주제로 다양한 활동을
# 하는 행사입니다.

# 대표로 파견된 인물에 대한 정보는 명확하지 않으며, 이러한 정보는 여러 나라의 참가자, 관계자,
# 정치인 등 다양하게 파견될 수 있습니다. 또한 축전의 성격과 목적에 따라 다양한 계층과 집단에서 대표가
# 파견되었을 가능성이 있습니다. 따라서, 구체적인 대표 인물을 지칭하기는 어렵습니다.
# 이러한 이벤트에서는 일반적으로 여러 나라의 청년 대표뿐만 아니라 정치인, 예술가,
# 사회운동가 등도 참가하기 때문에 대표 인물이 누구인지에 대한 정보는 매우 다양할 수 있습니다.

# GPT-4 맞춤유무 : x

In [ ]:
# idx 2
gen("임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?")
# 정답 : 1989년
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 임종석은 1989년 여의도 농민 폭력 시위를 주도한 혐의로 지명수배되었습니다.
# 그는 민주화 운동과 학생 운동에 활발하게 참여한 인물로, 여의도 시위 때의 역할에 대해 수사를 받았습니다.
# 이후에 그는 체포되어 재판을 받았고, 그 결과로 일정 기간을 복역한 뒤에 출소했습니다.
# 이 사건은 당시의 정치적, 사회적 상황 속에서 많은 논란을 불러일으켰습니다.

# 임종석은 후에 정치인으로 활동하며 다양한 역할을 했으며,
# 시간이 지남에 따라 그의 역할과 의미에 대한 평가도 다양해져 있습니다.

# GPT-4 맞춤유무 : o

In [ ]:
# idx 3
gen("임종석을 검거한 장소는 경희대 내 어디인가?")
# 정답 : 학생회관 건물 계단
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 지금까지의 정보로는 임종석은 1989년에 경희대학교 내에서 검거되었습니다.
# 그는 경희대학교의 도서관에서 검거되었다고 알려져 있습니다.
# 임종석은 당시 대학운동을 주도하던 인물로, 민주화 운동과 학생 운동에 앞장서 있었습니다.
# 그의 검거는 당시 정부와의 대립을 상징하는 사건 중 하나였습니다.

# 단, 이러한 정보는 시간이 지나면서 업데이트될 수 있으므로,
# 가장 최신의 정보를 확인하기 위해서는 신뢰할 수 있는 출처를 찾아보시는 것이 좋습니다.

# GPT-4 맞춤유무 : x

In [ ]:
# idx 4
gen("임종석이 조사를 받은 뒤 인계된 곳은 어딘가?")
# 정답 : 서울지방경찰청 공안분실
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 저의 마지막 업데이트 시점인 2021년 9월까지의 정보에 따르면, 임종석은 1989년에 민주화 운동에 참여한 혐의로 체포되었습니다.
# 그 후에는 서울 경찰청에서 조사를 받은 뒤 국가보안법 위반 등의 혐의로 기소되었습니다.

# 이러한 사건은 당시 대통령이었던 노태우 정부에 의해 실시되었으며,
# 임종석은 그 후 1991년에 석방되었습니다.
# 이후로 그는 민주화 운동과 관련된 다양한 활동을 계속하였고, 정치, 사회운동 등에서도 활발히 활동하였습니다.

# 자세한 정보는 시기와 상황에 따라 달라질 수 있으므로, 최신 정보를 확인하시기 바랍니다.

# GPT-4 맞춤유무 : x

In [ ]:
# idx 5
gen("1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?")
# 정답 : 임종석
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 1989년 2월 15일에 일어난 여의도 농민 폭력 시위와 관련하여 지명수배된 사람의 이름은 이용한(Lee Yong-han)입니다.
# 이 사건은 대한민국에서 농업 정책에 대한 논란과 농민들의 경제적 어려움을 둘러싼 여러 시위 중 하나였습니다.
# 이용한은 이 시위를 주도한 혐의로 지명수배되었으며, 이후에도 그는 농민 운동과 관련하여 여러 차례 논란의 중심에 있었습니다.

# 해당 정보는 제가 마지막으로 업데이트 된 시점까지의 내용이며, 더 최근의 정보가 있을 수 있습니다.

# GPT-4 맞춤유무 : x

In [ ]:
# idx 6
gen("임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?")
# 정답 : 여의도 농민 폭력 시위
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 1989년 2월 15일에 임종석이 지명수배 받은 혐의는 대한민국의 국가보안법 위반과
# 1987년에 발생한 '6월 항쟁'이라고 알려진 대규모 민주화 시위를 주도했다는 것이었습니다.
# 임종석은 당시에 대한민국에서 민주화 운동을 주도하는 주요 인물 중 하나였고,
# 그의 활동은 당시 정부에게는 국가보안법 위반으로 판단되었습니다.
# 6월 항쟁은 전국적인 민주화 운동으로 이어지며, 이후의 대한민국 정치에 큰 변화를 가져왔습니다.
# 이러한 활동 때문에 임종석은 지명수배를 받았고, 결국 1995년에 체포되어 재판을 받았습니다.

# 이렇게 설명하면 그의 활동이 오로지 불법적이고 반국가적인 것처럼 해석될 수 있지만,
# 그의 활동은 대한민국의 민주화 과정에서 중요한 역할을 했다고도 볼 수 있습니다.
# 시간이 흐르면서 대한민국은 민주화가 점점 더 이루어지고, 그 과정에서 임종석 같은
# 인물들이 어떠한 역할을 했는지에 대한 평가도 다양해졌습니다.

# GPT-4 맞춤유무 : x

In [ ]:
# idx 7
gen("정부의 헌법개정안 준비 과정에 대해서 청와대 비서실이 아니라 국무회의 중심으로 이뤄졌어야 했다고 지적한 원로 헌법학자는?")
# 정답 : 허영
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 제 지식의 마지막 업데이트는 2021년 9월까지이므로 그 이후의 사건이나 인물에 대한 정보는 제공할 수 없습니다.
# 그렇지만 대한민국에서 원로 헌법학자가 헌법개정안 준비 과정에 대하여 국무회의를 중심으로 이뤄져야 한다고
# 지적하는 것은 헌법 개정의 본질과 과정, 그리고 민주주의에 대한 학술적 논의를 반영할 수 있습니다.

# 헌법 개정이나 중요한 정책 결정에서 국무회의의 역할을 강조하는 것은 다양한 의견과 분야의
# 전문가들이 참여하는 통한 투명성과 책임성을 높이기 위한 것일 수 있습니다.
# 이런 지적은 보통 헌법학, 정치학, 행정학 등 다양한 학문에서 다루어지는 주제입니다.

# 만약 특정 원로 헌법학자가 이러한 지적을 한 것이 최근의 사안이라면, 해당 내용은 제가 알 수 없는 범위에 속하게 됩니다.
# 이 경우 신뢰할 수 있는 뉴스 출처나 학술 자료를 참고하시는 것이 좋을 것입니다.

# GPT-4 맞춤유무 : x

In [ ]:
# idx 8
gen("행보가 비서 본연의 역할을 벗어난다', '장관들과 내각이 소외되고 대통령비서실의 권한이 너무 크다'는 의견이 제기된 대표적인 예는?")
# 정답 : 10차 개헌안 발표
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# "행보가 비서 본연의 역할을 벗어난다", "장관들과 내각이 소외되고 대통령비서실의 권한이 너무 크다"는
# 비판은 일반적으로 대통령 혹은 총리 주변의 스태프나 비서실이 과도한 권력을 행사할 때 나타납니다. 이러한 현상은 여러 나라와
# 다양한 정부 체제에서 발생할 수 있으며, 특히나 대통령제가 시행되는 국가에서 더욱 두드러지곤 합니다.

# 미국: 미국에서는 이러한 비판이 때때로 "확장된 행정부(Imperial Presidency)"라는 용어로 불립니다.
# 특히 리처드 닉슨, 조지 W. 부시, 바락 오바마 대통령 등이 이러한 비판을 받은 경우가 있습니다.
# 대한민국: 대한민국에서도 대통령비서실의 권한과 역할이 너무 커진 것이 아니냐는 지적이 종종 있습니다.
# 이는 비서실이 실질적인 결정 권한을 행사하기도 하고, 장관이나 기타 고위 공직자보다도 대통령에게 더 가까운 영향력을 행사할 때 나타납니다.
# 프랑스: 프랑스의 '에리제 팔레스(Elysée Palace)'도 비슷한 비판을 받을 때가 있습니다.
# 프랑스 대통령은 헌법상으로 많은 권한을 가지고 있고, 때로는 이를 비서실을 통해 행사하기도 합니다.
# 러시아: 러시아에서도 크렘린(Kremlin) 내부의 스태프나 비서가 과도한 권력을 행사하는 것으로 비판받는 경우가 있습니다.

# 이러한 비판은 대체로 대통령이나 총리, 그리고 그들의 주변 스태프가
# '체크 앤 밸런스(checks and balances)'의 원칙을 무시하거나 약화시킬 때 나타나곤 합니다.
# 이는 민주주의 체제에서 논란이 될 수 있는 문제로, 특히나 대통령이나 총리가
# 독단적인 결정을 내릴 가능성이 있을 때 중요한 문제로 다루어집니다.

# GPT-4 맞춤유무 : x

In [ ]:
# idx 9
gen("국무회의의 심의를 거쳐야 한다는 헌법 제 몇 조의 내용인가?")
# 정답 : 제89조
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 한국 헌법에 따르면 국무회의의 심의를 거쳐야 하는 사항에 대한 규정은 여러 조에 걸쳐 나타나 있습니다.
# 그러나 대통령의 권한과 국무회의의 역할에 대해 구체적으로 다루고 있는 부분은 대체로 헌법 제89조 이후에 나옵니다.
# 이러한 내용은 헌법 개정이나 변경이 있을 수 있으므로, 정확한 정보를 원하신다면
# 가장 최근의 한국 헌법을 참고하시는 것이 좋을 것 같습니다.

# GPT-4 맞춤유무 : o

In [ ]:
# idx 10
gen("법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다고 지적한 경희대 석좌교수 이름은?")
# 정답 : 허영
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 죄송합니다만, 2021년 9월까지의 정보를 바탕으로는 그러한 사례나 경희대 석좌교수의 이름에 대한 정보를 확인할 수 없습니다.
# 정보가 업데이트되지 않았기 때문에 최신 사항에 대해 알려 드릴 수 없습니다.
# 다만, 일반적으로 법무부 장관이 아닌 민정수석이 법안 개정과 관련한 설명을 하는 경우에는 특별한 이유나 배경이 있을 수 있습니다.
# 이는 정치적 또는 행정적 판단에 따라 다를 수 있으므로, 그 배경과 이유에 대해
# 정확한 정보를 얻기 위해서는 관련 뉴스나 공식 발표를 참고하는 것이 좋을 것 같습니다.

# GPT-4 맞춤유무 : x

In [ ]:
# idx 11
gen("미국 군대 내 두번째로 높은 직위는 무엇인가?")
# 정답 : 미국 육군 부참모 총장
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 2021년 기준으로, 미국 군대 내에서 두 번째로 높은 직위는 여러 분야와 종류에 따라 다르지만,
# 일반적으로는 '부참모 총장' (Vice Chairman of the Joint Chiefs of Staff)이라고 할 수 있습니다.
# '참모 총장' (Chairman of the Joint Chiefs of Staff)이 미국 군대의 가장 높은 직위인데,
# 이는 육군, 해군, 공군, 해병대의 참모 총장과 국토경비대의 지휘관들이 모여
# 구성되는 고위 직위 그룹인 '합동참모본부' (Joint Chiefs of Staff)의 리더입니다.

# 부참모 총장은 참모 총장이 부재할 경우나 그 외의 상황에서 참모 총장의 역할을 대신하게 됩니다.
# 그 외에도 특별한 임무와 책임이 있을 수 있으며, 이는 현재 상황과 정책에 따라 달라질 수 있습니다.

# 미국의 각 군별로도 두 번째로 높은 직위가 있을 수 있는데, 이는 군 별로 다르고 특별한 경우에 따라 다를 수 있습니다.
# 예를 들어, 육군에서는 '부참모 총장' (Vice Chief of Staff), 해군에서는 '부참모' (Vice Chief of Naval Operations),
# 공군에서는 '부참모 총장' (Vice Chief of Staff), 해병대에서는 '부사령관' (Assistant Commandant) 등이 있을 수 있습니다.

# GPT-4 맞춤유무 : o

In [ ]:
# idx 12
gen("로널드 레이건 정부 출범 당시 알렉산더 헤이그는 어떤 직책을 맡았는가?")
# 정답 : 초대 국무장관직
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 로널드 레이건 대통령이 1981년에 취임했을 때 알렉산더 헤이그(Alexander Haig)는
# 미국 국무장관(Secretary of State)으로 임명되었습니다. 알렉산더 헤이그는 미국 육군 장군 출신으로,
# 리처드 닉슨과 제럴드 포드 대통령 아래에서 백악관 비서실장(White House Chief of Staff)을 지낸 경력도 있습니다.
#
# 레이건 정부에서 그는 주로 대외 정책과 외교 문제를 총괄하였으나,
# 그의 임기는 비교적 짧게 끝났고, 1982년에 사임하게 되었습니다.

# GPT-4 맞춤유무 : o

In [ ]:
# idx 13
gen("알렉산더 헤이그는 어느 대통령의 밑에서 국무장관을 지냈는가?")
# 정답 : 로널드 레이건 대통령
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 알렉산더 헤이그(Alexander Haig)는 로널드 레이건(Ronald Reagan) 대통령 밑에서
# 1981년부터 1982년까지 미국의 국무장관을 지냈습니다.
# 알렉산더 헤이그는 이전에도 리처드 닉슨(Richard Nixon) 및
# 제럴드 포드(Gerald Ford) 대통령 아래에서 백악관 비서실장을 역임했습니다.
# 그는 또한 군사 경력도 있으며, 미국 육군에서 4성 장군까지 승진한 이력이 있습니다.
# 헤이그는 국제 정치와 미국 내 정치에서 다양한 역할을 했으며, 그의 경력은 매우 다채로웠습니다.

# GPT-4 맞춤유무 : o

In [ ]:
# idx 14
gen("로널드 레이건 대통령 밑에서 일한 국무 장관은 누구인가?")
# 정답 : 알렉산더 메이그스 헤이그 2세
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 로널드 레이건 대통령이 집권했던 1981년부터 1989년까지 두 명의 국무 장관이 있었습니다.

# 1. 알렉산더 헤이그 (Alexander Haig) - 1981년 1월 22일부터 1982년 7월 5일까지 국무 장관을 지냈습니다.
# 그는 레이건 정부 초기에 재임했으며, 그 전에는 유엔 군사 사령관 및 닉슨 및 포드 정부에서 다양한 역할을 했습니다.

# 2. 조지 P. 슐츠 (George P. Shultz) - 1982년 7월 16일부터 1989년 1월 20일까지 재임했습니다.
# 슐츠는 레이건 정부에서 상당한 영향력을 가진 인물로, 미국과 소비에트 연합 간의 관계 개선과 미국의 중동 정책 등에 큰 역할을 했습니다.

# 이 두 장관은 레이건 대통령 밑에서 국무 장관을 역임하면서 미국의 외교 정책에 큰 영향을 끼쳤습니다.

# GPT-4 맞춤유무 : o

In [ ]:
# idx 15
gen("미국 군대에서 두번째로 높은 직위는?")
# 정답 : 미국 육군 부참모 총장
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 2021년 9월까지의 정보에 따르면, 미국 군대에서 두 번째로 높은 직위는 '부참모총장' (Vice Chairman of the Joint Chiefs of Staff)입니다.
# 부참모총장은 미국 합동참모본부의 참모총장을 보좌하며, 참모총장이 결석하거나 부재 중일 때 그 역할을 대신합니다.
# 부참모총장은 미국 군대의 모든 분야에서 일어나는 작전과 전략, 훈련에 관한 권한을 가지고 있습니다.

# 미국의 군사 지휘 구조에서 참모총장 (Chairman of the Joint Chiefs of Staff)은 가장 높은 군사 직위로,
# 대통령과 국방부 장관에게 군사 상의를 제공합니다. 참모총장이 가장 높은 직위이며,
# 그 아래에 부참모총장이 위치한다고 볼 수 있습니다.

# 주의할 점은 각 군의 최고 직위 (예: 육군의 '육군참모총장', 해군의 '해군작전총장', 공군의 '공군참모총장' 등)도
#  높은 직위이지만, 이들은 합동참모본부 참모총장 아래에 위치한다고 볼 수 있습니다.

# GPT-4 맞춤유무 : o

In [ ]:
# idx 16
gen("알렉산더 메이그스 헤이그의 생년월일은?")
# 정답 : 1924년 12월 2일
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 알렉산더 메이그스 헤이그(Alexander Meigs Haig Jr.)는 1924년 12월 2일에 태어났습니다.
# 그는 미국의 군인과 정치인으로, 로널드 레이건 대통령 밑에서 국무장관을 역임했습니다.
# 또한 리처드 닉슨과 제럴드 포드 대통령 아래에서 백악관 비서실장을 맡았으며, 미국 육군에서 4성 장군까지 올랐습니다.
# 헤이그는 2010년 2월 20일에 사망했습니다.

# GPT-4 맞춤유무 : o

In [ ]:
# idx 17
gen("알렉산더 헤이그가 로널드 레이건 대통령 밑에서 맡은 직책은 무엇이었나?")
# 정답 : 국무장관
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 알렉산더 헤이그(Alexander Haig)는 로널드 레이건(Ronald Reagan) 대통령 밑에서
# 1981년부터 1982년까지 미국 국무장관(Secretary of State)을 지냈습니다.
# 헤이그는 레이건 대통령이 취임한 이후 초기의 외교 정책을 담당하며 중요한 역할을 했습니다.
# 하지만 그의 임기는 상대적으로 짧았고, 내부에서의 정치적 긴장과 정책적 차이 때문에 1982년에 사임했습니다.

# 특히 알렉산더 헤이그는 1981년 레이건 대통령이 총격을 당한 사건 이후 "나는 여기서 권력을 쥐고 있다"라고 발언한 것으로 유명합니다.
# 이 발언은 다소 논란이 되었으며 헤이그의 정치적 경력에 영향을 미쳤습니다.
# 이는 헤이그가 대통령의 위기 상황에서 어떤 역할을 하려고 했는지,
# 그리고 그의 권력을 어떻게 이해하고 있었는지에 대한 논란을 촉발했습니다.

# GPT-4 맞춤유무 : o

In [ ]:
# idx 18
gen("알렉산더 헤이그가 1984년 발간한 회고록의 제목은 무엇인가?")
# 정답 : 경고:현실주의, 레이건과 외교 정책
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 알렉산더 헤이그의 1984년에 발간된 회고록의 제목은 "Caveat: Realism, Reagan, and Foreign Policy" 입니다.
# 이 책에서 헤이그는 그의 생각과 경험을 공유하며,
# 특히 로널드 레이건 정부에서의 국무장관 시절을 중심으로 다양한 외교 정책 이슈에 대해 논의합니다.

# GPT-4 맞춤유무 : o

In [ ]:
# idx 19
gen("알렉산더 헤이그와 1950년 5월 결혼한 상대의 이름은 무엇인가?")
# 정답 : 퍼트리샤 앤토이넷 폭스
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 알렉산더 헤이그(Alexander Haig)는 1950년 5월에 패트리샤 폭스(Patricia Fox)와 결혼했습니다.
# 알렉산더 헤이그는 미국의 군인이자 정치가로, 로널드 레이건 대통령 밑에서 국무장관을 역임했습니다.
# 그는 또한 리처드 닉슨과 제럴드 포드 대통령 밑에서 백악관 비서실장을 역임하기도 했습니다.
# 패트리샤 폭스와의 결혼을 통해 그는 여러 명의 자녀를 두었습니다.

# GPT-4 맞춤유무 : o

# 예측결과 성능 정리

In [ ]:
# 맞춘 개수 정리
# GPT-4(ChaptGPT) : 11/20
# Fine-Tuend Llama 3.2 1B(Data Augmentation 적용 o) : 7/20
# Fine-Tuend Llama 3.2 3B(Data Augmentation 적용 o) : 11/20